<a href="https://colab.research.google.com/github/RIPS-2024-Aerospace/Aerospace-Project/blob/main/MSE_Minimization_WithBFGS_LTT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/RIPS-2024-Aerospace/Aerospace-Project.git

Cloning into 'Aerospace-Project'...
remote: Enumerating objects: 489, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 489 (delta 143), reused 93 (delta 61), pack-reused 269
Receiving objects: 100% (489/489), 31.46 MiB | 12.62 MiB/s, done.
Resolving deltas: 100% (233/233), done.


In [30]:
import numpy as np
import random
import math
import scipy as sp

np.random.seed(29)

%run "/content/Aerospace-Project/DiffusionLunarKF.ipynb"
%run "/content/Aerospace-Project/CentralizedLunarKF.ipynb"
%run "/content/Aerospace-Project/FilterComparison.ipynb"


In [31]:
#Generates a random edge weighting for Lunar network
#Output is vector of size 8, weights correspond to edges: [(1,2), (1,3), (1,4), (1,5), (2,1), (3,1), (4,1), (5,1)]
def gen_first_guess():
  weights = []
  temp = []
  for i in range(5):
    temp.append(random.random())
  weights = [w/sum(temp) for w in [temp[i] for i in range(4)]]
  for j in range(4):
    edge_node = [random.random()]
    weights += edge_node
  return weights

In [32]:
#Converts len=8 vector to full adjacency matrix for use in filtering
def convert_to_adj(weights):
  network = []
  temp = []
  for i in range(4):
    temp.append(weights[i])
  network.append(1-sum(temp))
  network += temp
  for i in range(4, len(weights)):
    network.append(weights[i])
    network.append(1-weights[i])



  ref = np.array([[0.2,0.2,0.2,0.2,0.2], [0.5, 0.5, 0, 0,0], [0.5, 0, 0.5, 0,0], [0.5,0,0, 0.5,0],[0.5,0,0,0,0.5]]) #This just serves as reference for where there are edges
  W = []
  i = 0
  for arr in ref:
    for val in arr:
      if val != 0:
        W.append(network[i])
        i += 1
      else: W.append(0)
  adj_matrix = np.array([[W[i] for i in range(0,5)], [W[i] for i in range(5,10)], [W[i] for i in range(10,15)], [W[i] for i in range(15,20)], [W[i] for i in range(20,25)]])
  return(adj_matrix)

In [54]:
def MSE(vec):
    adj = np.array([[1,1,1,1,1],[1,1,0,0,0],[1,0,1,0,0],[1,0,0,1,0],[1,0,0,0,1]])

    C = convert_to_adj(vec)
    n = len(C)

    D = 3*adj
    D_ckf = 3*np.ones((n,n))

    np.random.seed(29)
    true_biases = np.array([[np.random.normal(0,np.sqrt(12/(c**2))) for _ in range(n)]]).T
    true_drifts = np.array([[np.random.normal(0,np.sqrt(0.1/(c**2))) for _ in range(n)]]).T
    # true_drifts = np.array([[0 for _ in range(n)]]).T

    F = np.array([[1,dt],[0,1]])
    F_full = np.kron(np.eye(n),F)

    x = c*np.vstack(tuple([np.array([true_biases[i],true_drifts[i]]) for i in range(n)]))

    # random initial estimates for each node

    x0 = [np.array([[np.random.normal(0,np.sqrt(12))],[np.random.normal(0,np.sqrt(0.1))]]) for i in range(n)]
    x0_cf = np.vstack(tuple(x0))
    # x0 = [np.array([[0],[0]]) for _ in range(n)]

    P = [100*np.copy(R(1)) for _ in range(n)]
    P_prev = np.block([[P[i] if i==j else np.zeros((2,2)) for j in range(n)] for i in range(n)])

    stations = [Station(i) for i in range(n)]

    filter_initialize(stations,D,x0,P)

    Q_10x10 = np.kron(np.eye(n),Q)

    kf = KalmanFilter(A = F_full, H = H_cf, Q = Q_10x10, R = R_cf, P = P_prev, x0 = x0_cf)

    iterations = 100

    # num_msmts = np.random.randint(0,10,(iterations,5))
    num_msmts = np.array([[5,5,5,5,5] for _ in range(iterations)])

    filter_outputs = run_both_filters(iterations, num_msmts,C,F_full,stations,kf, x)

    errors_df,errors_cf,P_hist_cf,P_hist_df,truth,measurements,predictions_cf,predictions_df = filter_outputs
    squared_error_iteration = lambda i: sum([x**2 for x in errors_df[i,:,0,0]])
    return sum([x**2 for x in [squared_error_iteration(i) for i in range(iterations)]])/(n*iterations)

In [56]:
#Run BFGS for LTT
def mse_bfgs():

  #Linear Constraints:
  A = np.array([[1,0,0,0,0,0,0,0],
              [0,1,0,0,0,0,0,0],
              [0,0,1,0,0,0,0,0],
              [0,0,0,1,0,0,0,0],
              [1,1,1,1,0,0,0,0],
              [0,0,0,0,1,0,0,0],
              [0,0,0,0,1,0,0,0],
              [0,0,0,0,0,1,0,0],
              [0,0,0,0,0,1,0,0],
              [0,0,0,0,0,0,1,0],
              [0,0,0,0,0,0,1,0],
              [0,0,0,0,0,0,0,1],
              [0,0,0,0,0,0,0,1]])
  cons = sp.optimize.LinearConstraint(A, lb = np.zeros(13), ub = np.ones(13))
  bounds = sp.optimize.Bounds(lb = np.zeros(8), ub = np.ones(8))

  #Initial Guess:
  x0 = gen_first_guess()

  result = sp.optimize.minimize(fun=MSE, x0=x0, bounds=bounds, constraints=cons)

  print("Optimized Weights Vector: ", np.around(result.x, 4))
  print("\nCorresponding Adjacency Matrix:")
  print(convert_to_adj(np.around(result.x, 4)))
  print("\nResulting Bhattacharya Distance: ", round(result.fun, 4))


mse_bfgs()

Optimized Weights Vector:  [0.0061 0.087  0.0214 0.2857 0.0942 0.0201 0.1519 0.0583]

Corresponding Adjacency Matrix:
[[0.5998 0.0061 0.087  0.0214 0.2857]
 [0.0942 0.9058 0.     0.     0.    ]
 [0.0201 0.     0.9799 0.     0.    ]
 [0.1519 0.     0.     0.8481 0.    ]
 [0.0583 0.     0.     0.     0.9417]]

Resulting Bhattacharya Distance:  3.8655
